In [ ]:
import cv2
import numpy as np
# from IPython.display import clear_output
import matplotlib.image as mpimg


from shapely.geometry import Polygon

import torch
from ultralytics import YOLO
from PIL import Image

In [ ]:
device = torch.device('cuda:0')

In [ ]:
def add_alpha_channel(image):
    """
    Add an alpha channel to a BGR image.
    
    Args:
        image (np.ndarray): Input image (H, W, 3).
    
    Returns:
        np.ndarray: Image with alpha channel (H, W, 4).
    """
    # Add a fully opaque alpha channel
    bgr = image
    alpha = np.ones(bgr.shape[:2], dtype=bgr.dtype) * 255  # Fully opaque
    return cv2.merge((bgr, alpha))

def apply_mask_with_alpha(image, mask, new_color=(0, 255, 0), alpha=0.5):
    """
    Apply a mask to an image with transparency.
    
    Args:
        image (np.ndarray): Input image with alpha channel (H, W, 4).
        mask (np.ndarray): Binary mask (H, W) with 255 for regions to modify.
        new_color (tuple): RGB color to apply.
        alpha (float): Transparency level (0.0 = transparent, 1.0 = opaque).
    
    Returns:
        np.ndarray: Image with modified regions and transparency applied.
    """
    # Split the image into BGR and alpha channels
    b, g, r, a = cv2.split(image)
    
    # Convert the new color to individual channels
    new_color_b, new_color_g, new_color_r = new_color
    
    # Apply the mask to the BGR channels
    b[mask == 255] = (1 - alpha) * b[mask == 255] + alpha * new_color_b
    g[mask == 255] = (1 - alpha) * g[mask == 255] + alpha * new_color_g
    r[mask == 255] = (1 - alpha) * r[mask == 255] + alpha * new_color_r
    
    # Combine the modified BGR channels with the original alpha channel
    return cv2.merge((b, g, r, a))


def yolo_to_mask(yolo_file, image_shape):
    """
    Generate a mask from YOLO label file.
    
    Args:
        yolo_file (str): Path to YOLO labels file.
        image_shape (tuple): Shape of the image (height, width).
    
    Returns:
        np.ndarray: Binary mask.
    """
    # Image dimensions
    height, width = image_shape[:2]

    # Initialize a blank mask
    mask = np.zeros((height, width), dtype=np.uint8)

    # Read YOLO file
    with open(yolo_file, 'r') as f:
        polygons = [line.strip().split() for line in f.readlines()]

    # Process each label (polygon)
    for poly in polygons:
        coords = list(map(float, poly[1:]))
        points = np.array([
            [int(coords[i] * width), int(coords[i + 1] * height)]
            for i in range(0, len(coords), 2)
        ], dtype=np.int32)

        # Draw the polygon on the mask
        cv2.fillPoly(mask, [points], 255)

    return mask

def calculate_polygon_iou(poly1_coords, poly2_coords):
    poly1 = Polygon(poly1_coords)
    poly2 = Polygon(poly2_coords)
    
    if not poly1.is_valid or not poly2.is_valid:
        return 0
    
    intersection_area = poly1.intersection(poly2).area
    union_area = poly1.union(poly2).area
    return intersection_area / union_area if union_area != 0 else 0

def read_yolo_polygons(file_path, image_width, image_height):
    polygons = []
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split()
            # Ignorar el class_id (primer valor)
            coords = list(map(float, data[1:]))
            # Convertir coordenadas normalizadas a coordenadas absolutas
            absolute_coords = [
                (int(x * image_width), int(y * image_height))
                for x, y in zip(coords[0::2], coords[1::2])
            ]
            polygons.append(absolute_coords)
    return polygons

def calculate_precision_yolo(predictions, ground_truth_file, image_width, image_height, iou_threshold=0.5):
    # Leer etiquetas del archivo
    ground_truth = read_yolo_polygons(ground_truth_file, image_width, image_height)
    
    # Emparejar predicciones con ground truth
    matched = []
    for pred in predictions:
        best_iou = 0
        best_gt = None
        for gt in ground_truth:
            iou = calculate_polygon_iou(pred, gt)
            if iou > best_iou:
                best_iou = iou
                best_gt = gt
        # Asociar si el IoU supera el umbral
            
        if best_iou >= iou_threshold:
            matched.append((pred, best_gt, best_iou))
            ground_truth.remove(best_gt)  # Evitar emparejamientos múltiples
    
    # Calcular métricas
    true_positives = len(matched)
    false_positives = len(predictions) - true_positives
    false_negatives = len(ground_truth)

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0

    return precision, recall

In [ ]:
image_path = "Put your image path here"
ground_truth_file = "Put your annotation file here"
model = YOLO("Path to weight.pt")

valid_p = []
invalid_p = []

## iterate 255 for each RGB channel
for Red in np.linspace(0,255, 16):
    for Green in np.linspace(0,255, 16):
        for Blue in np.linspace(0,255, 16):
            
            image = cv2.imread(image_path)
            
            # clear_output(wait=True)
            # print(image_path)
            # print(ground_truth_file)
            # print('\n\n\n',100*Red/255)
            # print(100*Green/255)
            # print(100*Blue/255,'\n\n\n')
            


            if image is None:
                raise FileNotFoundError("Image not found at the specified path")
            
            # Add an alpha channel
            image_with_alpha = add_alpha_channel(image)
            
            # Generate the mask
            mask = yolo_to_mask(ground_truth_file, image.shape)
            
            new_color = (Red, Green, Blue)  # Green
            BGR_new_color = (Blue, Green, Red)
            alpha = 0.7  # Transparency level
            modified_image = apply_mask_with_alpha(image_with_alpha, mask, new_color=BGR_new_color, alpha=alpha)
            
            
            rgb_image = cv2.cvtColor(modified_image, cv2.COLOR_BGRA2BGR)
            rgb_image = modified_image[:, :, :3]

            
            predictions = model.predict(source=rgb_image, save=False, imgsz=1536, conf=0.8, device=device, iou = 0)

            if(predictions[0].masks != None):
                # image_width = 2560
                # image_height = 1920
                image_width = 2048
                image_height = 1536
    
                precision, recall = calculate_precision_yolo(predictions[0].masks.xy, ground_truth_file, image_width, image_height)
                # print(f'P:{np.round(precision,1)} - R:{np.round(recall,1)}')
                valid_p.append([Red,Green,Blue, precision, recall])


            """
            If needed, filter by quality of precision and recall
            """
            #     if(precision >= 0.8 and recall >= 0.8):
            #         print('check')
            #         valid_p.append([Red,Green,Blue])
            #     else:
            #         invalid_p.append([Red,Green,Blue])
            # else:
            #     invalid_p.append([Red,Green,Blue])


save_valid = np.array(valid_p)
# save_invalid = np.array(invalid_p)


# np.savetxt("Pixel_PR/Biotite/012_pr.txt", save_valid, delimiter=",", header="R,G,B, Precision, Recall", comments="")